In [ ]:
import numpy as np
import pandas as pd
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from scipy.signal import savgol_filter
from sklearn.metrics import r2_score
from math import sqrt

# Load data from Excel file
data = pd.read_excel('nir.xlsx')    #update as needed

# Extract X (NIR data) and y (target values)
X = data.iloc[:, :-1].values  # update as needed
y = data[['Protein']].values  # Extracting the target columns and update as needed

# Preprocessing
# Baseline Correction (using Savitzky-Golay filter)
X_baseline_corrected = np.apply_along_axis(savgol_filter, axis=1, arr=X, window_length=15, polyorder=2)

# Normalization
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X_baseline_corrected)

# Vector Normalization
X_vector_normalized = X_normalized / np.linalg.norm(X_normalized, axis=1)[:, np.newaxis]

# First Derivative (assuming equally spaced data)
X_first_derivative = np.gradient(X_vector_normalized, axis=1)

# Calculate Z-scores for each sample
z_scores = np.abs((X_first_derivative - np.mean(X_first_derivative, axis=0)) / np.std(X_first_derivative, axis=0))

# Define threshold for Z-score
threshold = 3

# Identify outlier samples
outlier_samples_indices = np.where(np.any(z_scores > threshold, axis=1))[0]

# Remove outlier samples
X_first_derivative_filtered = np.delete(X_first_derivative, outlier_samples_indices, axis=0)
y_filtered = np.delete(y, outlier_samples_indices, axis=0)

# PLS Regression with filtered data
pls_model_filtered = PLSRegression(n_components=5)  # Choose the number of components

# Fit the PLS model with filtered data
pls_model_filtered.fit(X_first_derivative_filtered, y_filtered)

# Predicted values
y_pred_filtered = pls_model_filtered.predict(X_first_derivative_filtered)

# R-squared
r_squared_filtered = r2_score(y_filtered, y_pred_filtered)

# RMSECV
def calculate_rmsecv(model, X, y):
    y_pred = cross_val_predict(model, X, y, cv=5)
    rmsecv = sqrt(np.mean((y_pred - y) ** 2))
    return rmsecv

rmsecv_filtered = calculate_rmsecv(pls_model_filtered, X_first_derivative_filtered, y_filtered)

# RPD
def calculate_rpd(y_true, y_pred):
    std_dev = np.std(y_true)
    rpd = std_dev / sqrt(np.mean((y_pred - y_true) ** 2))
    return rpd

rpd_filtered = calculate_rpd(y_filtered, y_pred_filtered)

# Print evaluation metrics
print("R-squared after outlier removal:", r_squared_filtered)
print("RMSECV after outlier removal:", rmsecv_filtered)
print("RPD after outlier removal:", rpd_filtered)


import matplotlib.pyplot as plt

# Plot for SFA, MUFA, and PUFA
plt.figure(figsize=(14, 4))
# plt.suptitle('Pearl : Predicted vs. True', fontsize=16)  # Main heading for the plot

# Plot for SFA
plt.subplot(1, 3, 1)
plt.scatter(y_filtered[:, 0], y_pred_filtered[:, 0])
plt.plot(y_filtered[:, 0], y_filtered[:, 0], color='red', linestyle='--')
plt.xlabel('True Protein')  #update as needed
plt.ylabel('Predicted Protein') #update as needed
plt.title('Protein_Pearl: Predicted vs. True')  #update as needed
 
plt.show()